In [1]:
from urllib.request import urlopen
import json
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
import time
import random

from yahoofinancials import YahooFinancials
import math
from sklearn.manifold import MDS
from sklearn.neighbors import LocalOutlierFactor as LOF
from sklearn.svm import OneClassSVM
from scipy.stats import entropy
from matplotlib import pyplot as plt
from scipy.stats import energy_distance
import seaborn as sns
import datetime
import warnings
from scipy.special import kl_div
import csv
from datetime import date, timedelta
warnings.filterwarnings('ignore')

## Statistical metrics processing

1. Make a function to parse JSON files from https://financialmodelingprep.com/
2. Base on Apple ticket, retrieve the list of key statistics. The only one excluded - date.

In [2]:
def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/v3/company-key-metrics/AAPL?period=quarter")
apple = get_jsonparsed_data(url)

keys = []
for key in apple['metrics'][0]:
    if not key == 'date':
        keys.append(key)
keys

['Revenue per Share',
 'Net Income per Share',
 'Operating Cash Flow per Share',
 'Free Cash Flow per Share',
 'Cash per Share',
 'Book Value per Share',
 'Tangible Book Value per Share',
 'Shareholders Equity per Share',
 'Interest Debt per Share',
 'Market Cap',
 'Enterprise Value',
 'PE ratio',
 'Price to Sales Ratio',
 'POCF ratio',
 'PFCF ratio',
 'PB ratio',
 'PTB ratio',
 'EV to Sales',
 'Enterprise Value over EBITDA',
 'EV to Operating cash flow',
 'EV to Free cash flow',
 'Earnings Yield',
 'Free Cash Flow Yield',
 'Debt to Equity',
 'Debt to Assets',
 'Net Debt to EBITDA',
 'Current ratio',
 'Interest Coverage',
 'Income Quality',
 'Dividend Yield',
 'Payout Ratio',
 'SG&A to Revenue',
 'R&D to Revenue',
 'Intangibles to Total Assets',
 'Capex to Operating Cash Flow',
 'Capex to Revenue',
 'Capex to Depreciation',
 'Stock-based compensation to Revenue',
 'Graham Number',
 'Graham Net-Net',
 'Working Capital',
 'Tangible Asset Value',
 'Net Current Asset Value',
 'Invested Cap

In [3]:
SP500_symbols = ['AAPL', 'ABT', 'ABBV', 'ACN', 'ACE', 'ADBE', 'ADT', 'AAP', 'AES', 'AET', 'AFL', 'AMG', 'A', 'GAS', 'ARE', 'APD', 'AKAM', 'AA', 'AGN', 'ALXN', 'ALLE', 'ADS', 'ALTR', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'AIV', 'AMAT', 'ADM', 'AIZ', 'T', 'ADSK', 'ADP', 'AN', 'AZO', 'AVB', 'AVY', 'BHI', 'BLL', 'BAC', 'BK', 'BCR', 'BXLT', 'BAX', 'BBT', 'BDX', 'BBBY', 'BBY', 'BLX', 'HRB', 'BA', 'BWA', 'BXP', 'BSX', 'BMY', 'BRCM', 'CHRW', 'CA', 'CVC', 'COG', 'CAM', 'CPB', 'COF', 'CAH', 'HSIC', 'KMX', 'CCL', 'CAT', 'CBG', 'CBS', 'CELG', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHK', 'CVX', 'CMG', 'CB', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CTXS', 'CLX', 'CME', 'CMS', 'COH', 'KO', 'CCE', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CSC', 'CAG', 'COP', 'CNX', 'ED', 'STZ', 'GLW', 'COST', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DLPH', 'DAL', 'XRAY', 'DVN', 'DO', 'DTV', 'DFS', 'DISCA', 'DISCK', 'DLTR', 'D', 'DOV', 'DOW', 'DPS', 'DTE', 'DD', 'DUK', 'DNB', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMC', 'EMR', 'ENDP', 'ESV', 'ETR', 'EOG', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ES', 'EXC', 'EXPE', 'EXPD', 'ESRX', 'XOM', 'FFIV', 'FB', 'FAST', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FISV', 'FLIR', 'FLS', 'FLR', 'FMC', 'FTI', 'F', 'FOSL', 'BEN', 'FCX', 'FTR', 'GME', 'GPS', 'GRMN', 'GD', 'GE', 'GGP', 'GIS', 'GM', 'GPC', 'GNW', 'GILD', 'GS', 'GT', 'GOOGL', 'GOOG', 'GWW', 'HAL', 'HBI', 'HOG', 'HAR', 'HRS', 'HIG', 'HAS', 'HCA', 'HCP', 'HCN', 'HP', 'HES', 'HPQ', 'HD', 'HON', 'HRL', 'HSP', 'HST', 'HCBK', 'HUM', 'HBAN', 'ITW', 'IR', 'INTC', 'ICE', 'IBM', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IRM', 'JEC', 'JBHT', 'JNJ', 'JCI', 'JOY', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'GMCR', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KRFT', 'KR', 'LB', 'LLL', 'LH', 'LRCX', 'LM', 'LEG', 'LEN', 'LVLT', 'LUK', 'LLY', 'LNC', 'LLTC', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'MAC', 'M', 'MNK', 'MRO', 'MPC', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MAT', 'MKC', 'MCD', 'MCK', 'MJN', 'MMV', 'MDT', 'MRK', 'MET', 'KORS', 'MCHP', 'MU', 'MSFT', 'MHK', 'TAP', 'MDLZ', 'MON', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MUR', 'MYL', 'NDAQ', 'NOV', 'NAVI', 'NTAP', 'NFLX', 'NWL', 'NFX', 'NEM', 'NWSA', 'NEE', 'NLSN', 'NKE', 'NI', 'NE', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NRG', 'NUE', 'NVDA', 'ORLY', 'OXY', 'OMC', 'OKE', 'ORCL', 'OI', 'PCAR', 'PLL', 'PH', 'PDCO', 'PAYX', 'PNR', 'PBCT', 'POM', 'PEP', 'PKI', 'PRGO', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PBI', 'PCL', 'PNC', 'RL', 'PPG', 'PPL', 'PX', 'PCP', 'PCLN', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RRC', 'RTN', 'O', 'REGN', 'RF', 'RSG', 'RAI', 'RHI', 'ROK', 'COL', 'ROP', 'ROST', 'RLD', 'R', 'CRM', 'SNDK', 'SCG', 'SLB', 'SNI', 'STX', 'SEE', 'SRE', 'SHW', 'SPG', 'SWKS', 'SLG', 'SJM', 'SNA', 'SO', 'LUV', 'SWN', 'SE', 'STJ', 'SWK', 'SPLS', 'SBUX', 'HOT', 'STT', 'SRCL', 'SYK', 'STI', 'SYMC', 'SYY', 'TROW', 'TGT', 'TEL', 'TE', 'TGNA', 'THC', 'TDC', 'TSO', 'TXN', 'TXT', 'HSY', 'TRV', 'TMO', 'TIF', 'TWX', 'TWC', 'TJX', 'TMK', 'TSCO', 'RIG', 'TRIP', 'FOXA', 'TSN', 'TYC', 'UA', 'UNP', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'URBN', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VZ', 'VRTX', 'VIAB', 'V', 'VNO', 'VMC', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'ANTM', 'WFC', 'WDC', 'WU', 'WY', 'WHR', 'WFM', 'WMB', 'WEC', 'WYN', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XL', 'XYL', 'YHOO', 'YUM', 'ZION', 'ZTS']

In [4]:
# retrieve the statistical data for all SP500 companies 
data = []
for symbol in SP500_symbols:
    url = "https://financialmodelingprep.com/api/v3/company-key-metrics/{}?period=quarter".format(symbol)
    stock = get_jsonparsed_data(url)
    data.append(stock)
#     try:
#         print(symbol, len(stock['metrics']))
#     except:
#         pass

In [5]:
# save only those companies which has metrics
cleaned_data = []
for symbol in data:
    try:
        symbol['metrics']
        cleaned_data.append(symbol)
    except:
        pass

In [6]:
# save only those companies which has all 41 metric
full_info_data = []
for symbol in cleaned_data:
    try:
        if len(symbol['metrics']) > 30:
            full_info_data.append(symbol)
    except:
        pass

convert date from dictionary key-value representation, to the matrix of features
resulted `database` variable is 3-D. 
1. First dimention - the list of periods (quarterly)
2. Second dimention - the list of companies
3. Third - list of features described given company in a given period (taken from key metrics)

So, for example [0][2] - is an array of features for the company with index 2 in the first quartal

In [7]:
state_databases = []
for i in range(42):
    companies = []
    for symbol in full_info_data:
        company = []
        for key in keys:
            try:
                company.append(float(symbol['metrics'][i][key]))
            except:
                company.append(0)
        companies.append(company)
    state_databases.append(companies)

In [8]:
# in each quartal we need to define the kernal = distance matrix between companies. 
# For that firstly normalize the data, and then calculate rbf kernel
def kernalise_database(databases, norm="max", gamma=1):
    kernalizes_databases = []
    for base in databases:
        normal = normalize(np.array(base), norm="max")
        kernalizes_databases.append(rbf_kernel(normal, normal, gamma=gamma))
    return kernalizes_databases

In [9]:
# the list of all symbols which are in the database (~has full set of statistics in a given period of time)
keys_list = []
for symbol in full_info_data:
    keys_list.append(symbol['symbol'])
print(len(keys_list))

414


# Prices processing

In [10]:
def create_distance_matrix(historical_prices, approach='corr'):
    prices_matrix = historical_prices

# now prices_matrix - contains list of price changes for each company in a given period
# the next step - create a kernel (distance matrix) For that there are three possible approaches

# defaul one - correlation
    if approach=='corr':
        return np.absolute(np.corrcoef(prices_matrix))
    
#     entropy between two series
    if approach=='KL':
        distances_matrix = np.zeros((len(prices_matrix), len(prices_matrix)))
        for i, stock in enumerate(prices_matrix):
            for j, stock_2 in enumerate(prices_matrix):
                distances_matrix[i][j] = entropy(stock, qk=stock_2)
        return distances_matrix
    
#     custom - two companies closer when they have more corelated changed (the same direction of change in a day)
    if approach=='custom':
        distances_matrix = np.zeros((len(prices_matrix), len(prices_matrix)))
        for i, stock in enumerate(prices_matrix):
            for j, stock_2 in enumerate(prices_matrix):
                k = 0
                for q, price in enumerate(stock):
                    try:
                        if (price > 0 and stock_2[q] > 0) or (price <= 0 and stock_2[q] <= 0):
                            k += 1
                    except:
                        pass
                distances_matrix[i][j] = abs(k/len(stock) - 0.5) * 2 if len(stock) > 0 else 1
        return distances_matrix 

In [11]:
def get_prices(keys, start_date, end_date):
    price_matrix = []
    dates_matrix = []
    for key in keys:
        url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}".format(key, start_date, end_date)
        data = get_jsonparsed_data(url)
        try:
            price_matrix.append(list(map(lambda x: x['changePercent'], data['historical'])))
            dates_matrix.append(list(map(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").date(), data['historical'])))
        except:
            print(key, data)
    return price_matrix, dates_matrix[0]

In [12]:
def find_bounary_indexes(start_date, end_date, dates):
    index_a = 0
    index_b = 0
    j = 0
    while dates[j] < start_date:
        j+=1
    index_a = j

    while dates[j] < end_date:
        j+=1
    index_b = j + 1
    
    return index_a, index_b

In [13]:
# full_prices_list - 2D array, 1st dimenstion - companies, 2nd - prices from initial date to final
# dates - the list of stock exchange working dates
def load_prices_information(full_prices_list, dates):
    # original price matrixes (without normalisation and calculating the distance)
    price_original_matrixes = []
    # number of calendar days in each quartal
    intervals = [91, 91, 89, 90]
    # start date, and then with each step move it on corresponding number of days
    current_date = date(2009, 7, 1)
    # number of quartals
    number_of_intervals = 41
    
    index = 0
    for i in range(number_of_intervals):
    #   calculate start and end date for given № quartal
        start_date = ""
        end_date = ""
        increased_days = 0
    #   for leap year, there is one additional day in the first quartal
        if i == 10 or i == 26:
            increased_days = 1
        start_date = current_date
        end_date = (current_date + timedelta(days=intervals[i%4] + increased_days))
        current_date = current_date + timedelta(days=intervals[i%4] + 1 + increased_days)

        current_quartal = []
        
        low, high = find_bounary_indexes(start_date, end_date, dates)
        for company in full_prices_list:
            current_quartal.append(company[low: high])
            
        price_original_matrixes.append(current_quartal)

    return price_original_matrixes
    
    
# form distance matrixes from prices. Approach = 'corr'/'KL'/'custom'
# original matrixes - 3D quartal-company-prices
def calculate_price_distances(original_matrixes, approach='corr'):
        # matrixes of distances between companies based on price and default approach (correlation)
    price_distance_matrixes = []
    for i, matrix in enumerate(original_matrixes):
        distance_matrix = create_distance_matrix(historical_prices=matrix, approach=approach)
        price_distance_matrixes.append(distance_matrix)
    return price_distance_matrixes
    

In [14]:
# load raw prices matrixes
full_prices_list, dates = get_prices(keys_list, "2009-7-1", "2019-11-1")
price_original_matrixes = load_prices_information(full_prices_list, dates)

In [15]:
print(keys_list)

['AAPL', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AAP', 'AES', 'AFL', 'AMG', 'A', 'ARE', 'APD', 'AKAM', 'AGN', 'ALXN', 'ADS', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'AIV', 'AMAT', 'ADM', 'AIZ', 'T', 'ADSK', 'ADP', 'AN', 'AZO', 'AVB', 'AVY', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BBBY', 'BBY', 'HRB', 'BA', 'BWA', 'BXP', 'BSX', 'BMY', 'CHRW', 'COG', 'CPB', 'COF', 'CAH', 'HSIC', 'KMX', 'CCL', 'CAT', 'CBS', 'CELG', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHK', 'CVX', 'CMG', 'CB', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'CNX', 'ED', 'STZ', 'GLW', 'COST', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DO', 'DFS', 'DISCA', 'DLTR', 'D', 'DOV', 'DPS', 'DTE', 'DD', 'DUK', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENDP', 'ESV', 'ETR', 'EOG', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ES', 'EXC',

# Anomaly detection

In [16]:
# matrix: 2D array of distance measure of variables
# trevial approach based on mean distance 
# return the list of indexes with detected anomalies
def find_anomaly_custom(matrix, per_out, cloud_number=10):
    result = []
    mean_distance = np.mean(matrix) / 3
    # if cloud_number of nearest points located father than mean_distance, then it's anomaly  
    avg_distances = []
    for i, stock in enumerate(matrix):
        A = np.array(stock)
        idx = np.argpartition(A, cloud_number)
        avg_distances.append(np.mean(A[idx[:cloud_number]]))
        
    number_anomaly = int(per_out * len(matrix[0]))
    return np.argpartition(np.array(avg_distances), number_anomaly)[:number_anomaly]

# base on local outlier factor
def find_anomaly_lof(matrix, per_out):
    detector = LOF(metric='precomputed', contamination=per_out)
    inlines = detector.fit_predict(matrix)
    result = []
    for i, res in enumerate(inlines):
        if res == -1:
            result.append(i)
    return result

# based on SVM
def find_anomaly_svm(matrix, per_out):
    detector = OneClassSVM(kernel='precomputed', nu=per_out)
    inlines = detector.fit_predict(matrix)
    result = []
    for i, res in enumerate(inlines):
        if res == -1:
            result.append(i)
    return result

# use pre-computed matrix of distances!!
# general function to find outliers, approach one of ['custom', 'lof', 'svm']
def find_anomaly(matrix, approach, per_out, additional=10):
    if approach=='custom':
        return find_anomaly_custom(matrix, per_out, additional)
    if approach=='lof':
        return find_anomaly_lof(matrix, per_out)
    if approach=='svm':
        return find_anomaly_svm(matrix, per_out)

Combine two distance(kernel) matrixes. Statistical and price. 

stat_1, price_1 - training period.
stat_2, price_2 - test period

1. Apply sum or multiply operator for kernels. 
2. Based on new kernel, apply anomaly detector approach.
3. Compare two arrays of anomalieys. 
4. Return three values: the proporation of anomalies from the first (training) set which are also exist in the second (test) set, the number of detected anomalies in the first set and the list of symbols of intersections

In [17]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def get_anomalies_list(stat, price, detector, per_out, operation='sum', cloud_number=10):
    K = (stat + price) / 2
    if operation == 'mult':
        K = stat * price
    return find_anomaly(K, detector, per_out, cloud_number)

def indexes_to_symbol(indexes):
    symbols = []
    for i in indexes:
        symbols.append(keys_list[i])
    return symbols
    
def compare(stat_1, price_1, stat_2, price_2, detector, per_out, operation='sum', cloud_number=10):
    train_anomaly = get_anomalies_list(stat_1, price_1, detector, per_out, operation, cloud_number)
    test_anomaly = get_anomalies_list(stat_2, price_2, detector, per_out, operation, cloud_number)
    common = intersection(train_anomaly, test_anomaly)
    return len(common)/len(train_anomaly), len(train_anomaly), indexes_to_symbol(common)

In [18]:
print(np.shape(price_original_matrixes))

(41, 414)


In [19]:
# calculate and create a huge table of results
with open('resutls_8.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    steps = 10 #number of quarters to which we compare
    writer.writerow([
        'per_out', 
        'det-dist-oper', 
        'gamma',
        'cloud_number',
        'avg_len',
        *range(1, steps+1),
        *range(1, steps+1)])

#     'corr'/'custom'
    for distance_measure in ['custom']:
        price_matrixes = calculate_price_distances(price_original_matrixes, distance_measure)
#         'sum'/'mult'
        operation = 'sum'
        for gamma in np.arange(0.4, 0.6, 0.05):
            stat_matrixes = kernalise_database(state_databases, gamma=gamma)
#             'svm'/'lof'/'custom'
            for detector in ['custom']:
                for cloud_number in np.arange(10, 20, 5):
                    for per_out in np.arange(0.10, 0.13, 0.02):
                        step_means = []
                        step_std = []
                        avg_len = 0
                        for step in range(steps):
                            predicted_anomylies_percentage = []
                            predicted_length = []
                            for i in range(1, 40 - step):
                                a, b, c = compare(stat_matrixes[i-1], 
                                               price_matrixes[i], 
                                               stat_matrixes[i + step], 
                                               price_matrixes[i+1 + step],
                                               detector,
                                               per_out,
                                               operation,
                                                cloud_number)
                                predicted_anomylies_percentage.append(a)
                                predicted_length.append(b)
        #                     avg_len = np.mean(predicted_length)
                            avg_per = np.mean(predicted_anomylies_percentage)
                            avg_len = np.mean(predicted_length)
                            std = np.std(predicted_anomylies_percentage)
                            step_means.append(round(avg_per, 3))
                            step_std.append(round(std, 3))
                        writer.writerow([
                            round(per_out, 3), 
                            "{}-{}-{}".format(detector, distance_measure, operation),
                            round(gamma,2),
                            cloud_number,
                            round(avg_len,1),
                            *step_means,
                            *step_std
                        ]),

# exampled list of "true" anomalies
# 'AAP', 'ADS', 'AIG', 'ABC', 'COF', 'CB', 'CVS', 'DE', 'EMR', 'FITB', 'FCX', 'FTR', 'GNW', 'GILD', 'MCD', 'NTAP', 'NWL', 'PNR', 'PSA', 'PVH', 'SWN', 'TXN'